In [2]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
tf.get_logger().setLevel(tf.logging.ERROR)

from src.models import LeNet
from src.data_utils import get_mnist

print(f"GPU available: {tf.test.is_gpu_available()}")

(X_train, y_train), (X_test, y_test) = get_mnist()

# prepare the model
lenet = LeNet(
    input_shape=X_train.shape[1:],
    num_classes=10,
)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
GPU available: True


# Train

In [3]:
%%time

epochs=20
mc_rate=0.5
lenet.set_mc_dropout_rate(mc_rate)
lenet.train(X_train, y_train, X_test, y_test, epochs=epochs, verbose=1)

# lenet.save_model(f'lenet-{mc_rate}-{epochs}')

Training with mc_dropout_rate = 0.5.

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 6s 93us/step - loss: 0.1604 - acc: 0.9512 - val_loss: 0.0687 - val_acc: 0.9782
Epoch 2/20
60000/60000 [==============================] - 5s 85us/step - loss: 0.0680 - acc: 0.9795 - val_loss: 0.0603 - val_acc: 0.9803
Epoch 3/20
60000/60000 [==============================] - 7s 114us/step - loss: 0.0554 - acc: 0.9831 - val_loss: 0.0676 - val_acc: 0.9794
Epoch 4/20
60000/60000 [==============================] - 5s 89us/step - loss: 0.0469 - acc: 0.9858 - val_loss: 0.0553 - val_acc: 0.9846
Epoch 5/20
60000/60000 [==============================] - 5s 80us/step - loss: 0.0456 - acc: 0.9863 - val_loss: 0.0628 - val_acc: 0.9832
Epoch 6/20
60000/60000 [==============================] - 5s 83us/step - loss: 0.0375 - acc: 0.9887 - val_loss: 0.0552 - val_acc: 0.9870
Epoch 7/20
60000/60000 [==============================] - 6s 94us/step - loss: 0.0367 - a

In [4]:
from gym_eli.envs import EliEnv
import gym

from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2

mask = 1000
X_train = X_train[:mask]
y_train = y_train[:mask]

# The algorithms require a vectorized environment to run
env = DummyVecEnv([lambda: EliEnv(net=lenet,
                                  confidence_rate=0.5, 
                                  X_train=X_train, 
                                  y_train=y_train,
                                  max_mc_dropout_iterations=500,
                                  basic_option=False
                                  )])

in inittt


# Train Agent

In [5]:
%%time
model = PPO2(MlpPolicy, env, verbose=1, tensorboard_log="ppo2_tensorboard")
model.learn(total_timesteps=500)

# obs = env.reset()
# for i in range(2000):

action = 244, err = 0.002, reward = 997.512
action = 52, err = 0.001, reward = 998.948
action = 386, err = 0.002, reward = 997.228
action = 76, err = 0.002, reward = 997.848
action = 326, err = 0.002, reward = 997.348
action = 70, err = 0.002, reward = 997.86
action = 157, err = 0.002, reward = 997.686
action = 441, err = 0.002, reward = 997.118
action = 122, err = 0.002, reward = 997.756
action = 208, err = 0.001, reward = 998.792
action = 195, err = 0.002, reward = 997.61
action = 351, err = 0.002, reward = 997.298
action = 91, err = 0.002, reward = 997.818
action = 433, err = 0.002, reward = 997.134
action = 209, err = 0.002, reward = 997.582
action = 334, err = 0.002, reward = 997.332
action = 482, err = 0.002, reward = 997.036
action = 152, err = 0.002, reward = 997.696
action = 256, err = 0.002, reward = 997.488
action = 319, err = 0.001, reward = 998.681
action = 146, err = 0.002, reward = 997.708
action = 390, err = 0.002, reward = 997.22
action = 477, err = 0.002, reward = 997

In [6]:
%%time

env = DummyVecEnv([lambda: EliEnv(net=lenet,
                                  confidence_rate=0.5, 
                                  X_train=X_train, 
                                  y_train=y_train,
                                  max_mc_dropout_iterations=500,
                                  basic_option=True
                                  )])

model = PPO2(MlpPolicy, env, verbose=1, tensorboard_log="ppo2_tensorboard")
model.learn(total_timesteps=500)


in inittt
action = 446, err = 0.002, reward = 552.0
action = 287, err = 0.002, reward = 711.0
action = 189, err = 0.002, reward = 809.0
action = 216, err = 0.002, reward = 782.0
action = 341, err = 0.002, reward = 657.0
action = 412, err = 0.002, reward = 586.0
action = 83, err = 0.003, reward = 914.0
action = 61, err = 0.002, reward = 937.0
action = 64, err = 0.002, reward = 934.0
action = 477, err = 0.002, reward = 521.0
action = 439, err = 0.002, reward = 559.0
action = 437, err = 0.002, reward = 561.0
action = 299, err = 0.001, reward = 700.0
action = 395, err = 0.002, reward = 603.0
action = 112, err = 0.002, reward = 886.0
action = 460, err = 0.002, reward = 538.0
action = 36, err = 0.001, reward = 963.0
action = 122, err = 0.002, reward = 876.0
action = 319, err = 0.001, reward = 680.0
action = 121, err = 0.002, reward = 877.0
action = 57, err = 0.001, reward = 942.0
action = 455, err = 0.002, reward = 543.0
action = 248, err = 0.002, reward = 750.0
action = 18, err = 0.001, rew